# 1. RAG chain 구현 구문

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_community.tools import TavilySearchResults
from langchain_core.documents import Document

# 평가 알로리즘 모듈
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper




from textwrap import dedent
from operator import itemgetter

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
########################################################
# config 목록
########################################################
COLLECTION_NAME = "bluer_db_openai"
PERSIST_DIRECTORY = "vector_store/chroma/bluer_db"
EMBEDDING_MODEL_NAME = "text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)
MODEL_NAME = 'gpt-4o-mini'


########################################################
# vector_db에서 데이터 불러오기
########################################################

# vector store 연결
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

# 저장된 데이터 내용 확인
documents = vector_store._collection.get()['documents']
metadatas = vector_store._collection.get()['metadatas']

print(f"Documents: {documents[:1]}") 
print(f"Metadatas: {metadatas[:1]}")

Documents: ["{'Unnamed: 0': 0, 'id': 4627, '예약': '예약가능', '음식종류': '중식, 일반중식', '식당이름': '홍보각', '리본개수': 3, '전화번호': '02-531-6479', '영업시간_평일': '미등록', '영업종료_평일': '미등록', '영업시간_주말': '미등록', '영업종료_주말': '미등록', '휴무일': '연중무휴', '앱사용여부': True, '주차가능여부': '가능', '카드결제가능여부': 'y', '방문가능여부': nan, '메뉴정보': '스페셜코스(1인 15만원), 디너코스(A 12만원, B 16만원), 고법불도장(13만원), 모자새우(소 6만원, 대 10만원), 한알탕수육(소 4만8천원, 대 7만원), 특선냉채(2만8천원), 셰프특선수프(3만8천원), 셰프스페셜송이특찜(15만원)', '가격대': '10~15만원대', '영업시간': '12:00~15:00/18:00~21:30(마지막 주문 20:00) ', '신규오픈날짜': '2007년 ', '도로명주소': '서울특별시 강남구 봉은사로 130', '시': '서울특별시', '구': '강남구', '도로명': '동호로', '리뷰_상세': '국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스토랑. 고급스러운 분위기의 인테리어에서 정통 중식을 맛볼 수 있다. 여경래 셰프의 시그니처 메뉴인 불도장, 모자새우 등을 선보인다.', '리뷰_요약': '국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스토랑. 고급스러운 분위기의 인테리어에서 정통 중식을 맛볼 수 있다. 여경래 셰프의 시그니처 메뉴인 불도장, 모자새우 등을 선보인다.', '리뷰_평가': '정보 없음', '좌석_수': '64', '룸_수': '6개(2, 3, 4, 6, 8, 12석)', '체인점여부': False, '리뉴얼여부': False, '대표점여부': '정보 없음', '지점여부': '정보 없음', '웹사이트': 'https://w

In [3]:
print(f"Documents: {documents[:3]}") 

Documents: ["{'Unnamed: 0': 0, 'id': 4627, '예약': '예약가능', '음식종류': '중식, 일반중식', '식당이름': '홍보각', '리본개수': 3, '전화번호': '02-531-6479', '영업시간_평일': '미등록', '영업종료_평일': '미등록', '영업시간_주말': '미등록', '영업종료_주말': '미등록', '휴무일': '연중무휴', '앱사용여부': True, '주차가능여부': '가능', '카드결제가능여부': 'y', '방문가능여부': nan, '메뉴정보': '스페셜코스(1인 15만원), 디너코스(A 12만원, B 16만원), 고법불도장(13만원), 모자새우(소 6만원, 대 10만원), 한알탕수육(소 4만8천원, 대 7만원), 특선냉채(2만8천원), 셰프특선수프(3만8천원), 셰프스페셜송이특찜(15만원)', '가격대': '10~15만원대', '영업시간': '12:00~15:00/18:00~21:30(마지막 주문 20:00) ', '신규오픈날짜': '2007년 ', '도로명주소': '서울특별시 강남구 봉은사로 130', '시': '서울특별시', '구': '강남구', '도로명': '동호로', '리뷰_상세': '국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스토랑. 고급스러운 분위기의 인테리어에서 정통 중식을 맛볼 수 있다. 여경래 셰프의 시그니처 메뉴인 불도장, 모자새우 등을 선보인다.', '리뷰_요약': '국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스토랑. 고급스러운 분위기의 인테리어에서 정통 중식을 맛볼 수 있다. 여경래 셰프의 시그니처 메뉴인 불도장, 모자새우 등을 선보인다.', '리뷰_평가': '정보 없음', '좌석_수': '64', '룸_수': '6개(2, 3, 4, 6, 8, 12석)', '체인점여부': False, '리뉴얼여부': False, '대표점여부': '정보 없음', '지점여부': '정보 없음', '웹사이트': 'https://w

In [5]:
print(documents[0])

{'Unnamed: 0': 0, 'id': 4627, '예약': '예약가능', '음식종류': '중식, 일반중식', '식당이름': '홍보각', '리본개수': 3, '전화번호': '02-531-6479', '영업시간_평일': '미등록', '영업종료_평일': '미등록', '영업시간_주말': '미등록', '영업종료_주말': '미등록', '휴무일': '연중무휴', '앱사용여부': True, '주차가능여부': '가능', '카드결제가능여부': 'y', '방문가능여부': nan, '메뉴정보': '스페셜코스(1인 15만원), 디너코스(A 12만원, B 16만원), 고법불도장(13만원), 모자새우(소 6만원, 대 10만원), 한알탕수육(소 4만8천원, 대 7만원), 특선냉채(2만8천원), 셰프특선수프(3만8천원), 셰프스페셜송이특찜(15만원)', '가격대': '10~15만원대', '영업시간': '12:00~15:00/18:00~21:30(마지막 주문 20:00) ', '신규오픈날짜': '2007년 ', '도로명주소': '서울특별시 강남구 봉은사로 130', '시': '서울특별시', '구': '강남구', '도로명': '동호로', '리뷰_상세': '국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스토랑. 고급스러운 분위기의 인테리어에서 정통 중식을 맛볼 수 있다. 여경래 셰프의 시그니처 메뉴인 불도장, 모자새우 등을 선보인다.', '리뷰_요약': '국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스토랑. 고급스러운 분위기의 인테리어에서 정통 중식을 맛볼 수 있다. 여경래 셰프의 시그니처 메뉴인 불도장, 모자새우 등을 선보인다.', '리뷰_평가': '정보 없음', '좌석_수': '64', '룸_수': '6개(2, 3, 4, 6, 8, 12석)', '체인점여부': False, '리뉴얼여부': False, '대표점여부': '정보 없음', '지점여부': '정보 없음', '웹사이트': 'https://www.instagram.

In [4]:
from langchain_core.output_parsers import StrOutputParser
from textwrap import dedent

vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)


# GPT Model 생성
model = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0 
)


retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 100,
        "fetch_k": 200,
        "lambda_mult": 0.5,
        "filters": {"리본개수": {"$gte": 2}}
    }
)


# Prompt Template 생성
prompt_template = ChatPromptTemplate.from_messages([
    ("system", dedent("""
                당신은 한국의 식당을 소개하는 인공지능 비서입니다. 
                반드시 질문에 대해서 [context]에 주어진 내용을 바탕으로 답변을 해주세요. 
                만약 context에 질문과 관련된 내용이 없으면 "해당 정보가 없어 답변할 수 없습니다." 라고 대답해 주세요. 
                없는 정보를 만들어서 대답하지 마세요.
                # 만약 질문에 리본개수가 몇개인 식당 또는 몇개 이상, 이하인 식당에 대한 질문이 들어오면 [context] 식당 정보의 "리본개수" 항목을 확인해서 답변하세요.
                
                [context]
                {context}
                """)),
    ("human", "{question}")
])

# prompt_template = ChatPromptTemplate.from_messages([
#     ("system", "당신은 한국의 블루리본 서베이 전문가입니다. 질문에 자세히 답해주세요."),
#     ("human", "{question}")
# ])


# prompt_template = ChatPromptTemplate.from_messages([
#     ("system", """당신은 한국의 블루리본 서베이 안내자입니다. 주어진 데이터를 꼼꼼히 확인하여 가능한 정확한 정보를 주세요. 
#         # 반드시 context 의 내용을 바탕으로 대답해 주세요. 
#         '리본'과 관련된 모든 질문은 '리본이 몇 개인지'와 같은 숫자 조건을 포함한 검색으로 간주합니다.
#         '모든 질문에 대한 응답은 저장된 데이터를 토대로 정확한 갯수와 답변을 해주세요'
#         [context]
#         {context} """),
#     ("human", "{question}")
# ])




#########################################
# Chain 생성
#########################################

def content_from_doc(docs:list[Document]):
    return "\n\n".join([d.page_content for d in docs])


chain =  {'context': retriever  |RunnableLambda(content_from_doc), 'question': RunnablePassthrough()}  | prompt_template | model | StrOutputParser()

In [5]:
QUERY = "서대문구 근처 음식점 중 리본개수가 2개인 식당 다섯 곳을을 추천해주세요. 주소와 리본개수는 꼭 결과에 넣어주세요."
response = chain.invoke(QUERY)
print(response)

해당 정보가 없어 답변할 수 없습니다.


In [6]:
QUERY = "서대문구 근처 음식점 다섯 곳만 추천해주세요."
response = chain.invoke(QUERY)
print(response)

서대문구 근처 음식점 다섯 곳은 다음과 같습니다:

1. **형제집**
   - 음식종류: 한식주점
   - 전화번호: 02-921-3626
   - 주소: 서울특별시 동대문구 제기로6길 14-2
   - 메뉴: 닭볶음탕, 감자탕, 닭갈비 등

2. **또또**
   - 음식종류: 한식주점
   - 전화번호: 02-332-7312
   - 주소: 서울특별시 서대문구 홍제천로6길 2
   - 메뉴: 보쌈, 부대찌개, 차돌숙주볶음 등

3. **술익는마을**
   - 음식종류: 한식주점
   - 전화번호: 02-338-1190
   - 주소: 서울특별시 서대문구 연세로7길 22
   - 메뉴: 해물볶음우동, 소고기다타키 등

4. **한술식당(신촌점)**
   - 음식종류: 한식(일반한식), 덮밥
   - 전화번호: 0507-1479-8267
   - 주소: 서울특별시 서대문구 연세로4길 40
   - 메뉴: 간장삼겹덮밥, 오리지널곱창덮밥 등

5. **자주식당**
   - 음식종류: 한식주점
   - 전화번호: 02-511-8843
   - 주소: 서울특별시 강남구 강남대로156길 17
   - 메뉴: 고기요리, 해물요리 등

이 음식점들은 다양한 한식 메뉴를 제공하며, 서대문구와 인근 지역에서 인기 있는 곳들입니다.


In [7]:
QUERY = "리본개수가 2개인 식당을 추천해주세요"
response = chain.invoke(QUERY)
print(response)

리본개수가 2개인 식당은 다음과 같습니다:

1. **팔레드신**
   - 음식종류: 중식, 모던 차이니즈
   - 전화번호: 02-317-4001
   - 가격대: 10~15만원대
   - 도로명주소: 서울특별시 중구 퇴계로 67
   - 영업시간: 11:30~15:00/17:30~22:00
   - 웹사이트: [팔레드신 웹사이트](https://www.lescapehotel.com/dining/diner03)

2. **핑하오**
   - 음식종류: 중식, 일반중식
   - 전화번호: 02-3274-1188
   - 가격대: 2~5만원대
   - 도로명주소: 서울특별시 마포구 마포대로 45
   - 영업시간: 11:30~14:30/16:30~21:50
   - 웹사이트: 정보 없음

이 두 식당을 추천드립니다!


In [20]:
# 단순
QUERY = "리본개수가 2인 식당을 다섯 곳곳 추천해주세요"
response = chain.invoke(QUERY)
print(response)

리본개수가 2개인 식당은 다음과 같습니다:

- **식당이름**: 대려도
  - **가격대**: 5~10만원대
  - **구**: 강남구
  - **도로명주소**: 서울특별시 강남구 역삼로 118
  - **리뷰_상세**: 한국인의 입맛에 맞는 정통 북경요리를 선보이는 곳. 짜장면, 짬뽕 등의 식사 메뉴를 비롯해 냉채, 샥스핀, 고기요리 등 다채로운 요리를 즐길 수 있다. 코스메뉴도 가격 대비 높은 만족도를 자랑한다. 연회실과 룸을 별도로 갖추고 있어 모임 장소로도 제격이다.
  - **메뉴정보**: 삼선짜장면(1만2천원), 삼선짬뽕, 새우볶음밥(각 1만4천원), 크림중새우(소 4만7천원, 대 6만7천원), 깐풍기(소 3만9천원, 대 5만5천원), 점심코스(송 3만8천원, 죽 4만8천원), 저녁코스(일 6만원, 산 7만8천원, 강 12만 )
  - **영업시간**: 11:30~15:00/17:30~22:00
  - **전화번호**: 02-555-0550
  - **주차가능여부**: 발레 파킹
  - **휴무일**: 명절 휴무

이 식당은 고급스러운 분위기와 다양한 메뉴로 추천할 만합니다.


In [8]:
# 단순
QUERY = "리본개수가 2개인 식당을 몇 곳 추천해주세요"
response = chain.invoke(QUERY)
print(response)

리본개수가 2개인 식당은 다음과 같습니다:

1. **백리향**
   - 전화번호: 02-789-5741
   - 음식종류: 중식, 일반중식
   - 가격대: 15~25만원대
   - 도로명주소: 서울특별시 영등포구 63로 50
   - 웹사이트: [백리향 웹사이트](http://www.63restaurant.co.kr)

2. **팔레드신**
   - 전화번호: 02-317-4001
   - 음식종류: 중식, 모던차이니즈, 홍콩식중식, 북경오리
   - 가격대: 10~15만원대
   - 도로명주소: 서울특별시 중구 퇴계로 67
   - 웹사이트: [팔레드신 웹사이트](https://www.lescapehotel.com/dining/diner03)

3. **온지음레스토랑**
   - 전화번호: 02-6952-0024
   - 음식종류: 한식(모던한식)
   - 가격대: 15~25만원대
   - 도로명주소: 서울특별시 종로구 효자로 49
   - 웹사이트: [온지음레스토랑 웹사이트](https://www.instagram.com/onjium_restaurant/)

이렇게 3곳의 식당을 추천드립니다.


In [9]:
QUERY = "리본개수가 2개인 식당을 다섯 곳을 추천해주세요"
response = chain.invoke(QUERY)
print(response)

해당 정보가 없어 답변할 수 없습니다.


In [14]:
QUERY = "식당을 추천해주세요"
response = chain.invoke(QUERY)
print(response)

추천할 만한 식당을 몇 곳 소개해 드리겠습니다.

1. **대려도 (서울특별시 강남구)**
   - 음식종류: 중식
   - 가격대: 5~10만원대
   - 특징: 정통 북경요리를 선보이며, 다양한 요리를 즐길 수 있는 곳입니다. 연회실과 룸을 별도로 갖추고 있어 모임 장소로도 적합합니다.
   - 전화번호: 02-555-0550
   - 웹사이트: [대려도 웹사이트](http://www.daeryudo.com)

2. **목석원가든 (경상북도 안동시)**
   - 음식종류: 한식
   - 가격대: 2~5만원대
   - 특징: 하회마을 초입에 위치하며, 간고등어 정식과 안동찜닭이 유명합니다. 고즈넉한 분위기에서 식사를 즐길 수 있습니다.
   - 전화번호: 054-853-5332

3. **만강 (강원특별자치도 춘천시)**
   - 음식종류: 일식
   - 가격대: 5~10만원대
   - 특징: 고급 일식집으로 다양한 회를 맛볼 수 있으며, 코스 메뉴도 제공됩니다.
   - 전화번호: 033-262-5900
   - 웹사이트: [만강 웹사이트](http://mangang.ktib.co.kr/)

4. **청자골종가집 (전라남도 강진군)**
   - 음식종류: 한정식
   - 가격대: 2~5만원대
   - 특징: 40여 가지의 반찬이 푸짐하게 차려지는 한정식 전문점으로, 전통적인 분위기에서 식사를 즐길 수 있습니다.
   - 전화번호: 061-433-1100

이 외에도 다양한 식당이 있으니, 원하는 음식 종류나 분위기에 따라 선택하시면 좋겠습니다!


In [17]:
QUERY = "프랑스식을 추천해주세요"
response = chain.invoke(QUERY)
print(response)

프랑스식 요리를 즐길 수 있는 몇 가지 추천 식당을 소개해 드리겠습니다.

1. **오부이용**
   - **위치**: 서울특별시 성동구 독서당로51길 29-1
   - **추천 메뉴**: 양파수프, 파테, 부르고뉴식달팽이, 코코뱅, 저녁코스
   - **영업시간**: 12:00~15:00/18:00~22:00
   - **예약**: 예약 가능
   - **웹사이트**: [Instagram](https://www.instagram.com/aubouillon)

2. **르샹띠에**
   - **위치**: 경기도 고양시 일산동구 동판교로177번길 25
   - **추천 메뉴**: 셰프스페셜, 한우안심스테이크, 비프부르기뇽, 오리다리콩피
   - **영업시간**: 11:30~15:00/18:00~22:00
   - **예약**: 예약 가능
   - **웹사이트**: [Instagram](https://www.instagram.com/le.sentier/)

3. **꼼모아**
   - **위치**: 서울특별시 강남구 선릉로135길 29
   - **추천 메뉴**: 비프웰링턴, 오리다리콩피, 트러플화이트라구파스타
   - **영업시간**: 12:00~15:00/17:30~22:00
   - **예약**: 예약 가능
   - **웹사이트**: [Instagram](https://www.instagram.com/commemoa.gn/)

이 외에도 다양한 프랑스식 레스토랑이 있으니, 원하시는 분위기나 메뉴에 따라 선택하시면 좋겠습니다!


In [18]:
QUERY = "가족과 함께 갈만한 식당을 추천해주세요"
response = chain.invoke(QUERY)
print(response)

가족과 함께 가기 좋은 식당으로는 다음과 같은 곳들이 있습니다:

1. **신화정** (전라남도 순천시 구암길 26)
   - **음식종류**: 한정식
   - **특징**: 떡갈비를 메인으로 한 한정식을 제공하며, 정갈하고 깔끔한 스타일의 음식으로 상견례나 손님 접대 장소로 적합합니다. 2인, 3인, 4인 기준으로 한상이 차려지며 1인 반상도 주문 가능합니다.

2. **소곱친구** (경기도 수원시 영통구 효원로 393)
   - **음식종류**: 양곱창
   - **특징**: 깔끔한 분위기에서 한우 곱창과 대창, 막창을 전문으로 하며, 두꺼운 철판 위에 곱창을 올려 구워 먹습니다. 가족 단위로 방문하기 좋은 곳입니다.

3. **너른마당** (경기도 고양시 덕양구 서삼릉길 233-4)
   - **음식종류**: 한식(가금류)
   - **특징**: 넓은 마당에서 키운 닭을 사용한 백숙과 닭볶음탕이 인기입니다. 가족 단위로 편안하게 식사할 수 있는 공간이 마련되어 있습니다.

이 외에도 다양한 메뉴와 편안한 분위기를 제공하는 식당들이 많으니, 가족의 취향에 맞는 곳을 선택하시면 좋겠습니다!


## 기록

## 데이터 검토를 위한 코드들

In [20]:
# Retriever를 통해 관련 문서 가져오기
documents = retriever.get_relevant_documents(QUERY)

# 관련 문서 출력
for idx, doc in enumerate(documents):
    print(f"문서 {idx + 1}:")
    print("Metadata:", doc.metadata)
    print("Content:", doc.page_content)
    print("------")

C:\Users\Playdata\AppData\Local\Temp\ipykernel_25792\1666571792.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  documents = retriever.get_relevant_documents(QUERY)


문서 1:
Metadata: {}
Content: id: 35255
bookStatus: INACTIVE
foodTypes: 한식(육류), 소고기구이, 돼지고기구이
headerInfo_nameKR: 털보정육식당(장위2호점)
headerInfo_ribbonType: NOT_RECORD
defaultInfo_phone: 0507-1327-1979
defaultInfo_openHours: nan
defaultInfo_closeHours: nan
defaultInfo_openHoursWeekend: nan
defaultInfo_closeHoursWeekend: nan
defaultInfo_dayOff: 연중무휴
defaultInfo_app2Yn: False
statusInfo_parking: 가능
statusInfo_creditCard: nan
statusInfo_visit: nan
statusInfo_menu: 1++한우(변동), 한돈(변동), 한우탕밥(9천원), 한우육회비빔밥(1만2천원), 한우사골(1만원~2만원), 물냉면, 비빔냉면(각 7천원)
statusInfo_priceRange: 2~5만원대
statusInfo_businessHours: 12:00~15:00/17:00~23:00│토, 일요일 12:00~23:00
statusInfo_newOpenDate: nan
juso_roadAddrPart1: 서울특별시 성북구 돌곶이로 157
juso_siNm: 서울특별시
juso_sggNm: 성북구
juso_rn: nan
review_review: 원하는 소나 돼지고기 부위를 먼저 계산 후 구워 먹는 정육식당으로, 털보정육식당의 2호점. 1++ 등급 한우 암소의 다양한 부위를 맛볼 수 있으며, 한 마리 세트도 있다. 숯불에 구워 먹으며, 밑반찬과 된장찌개, 육회도 내어 준다. 
review_reviewSimple: 원하는 소나 돼지고기 부위를 먼저 계산 후 구워 먹는 정육식당으로, 털보정육식당의 2호점. 1++ 등급 한우 암소의 다양한 부위를 맛볼 수 있으며, 한 마

## 잡것들

In [ ]:
# retrieval_qa = RetrievalQA.from_chain_type(
#     llm=model,
#     retriever=retriever,
#     return_source_documents=True,  # Include source documents in response
# )
# response = retrieval_qa({"query": QUERY})

# print("응답:", response["result"])